In [1]:
import pandas as pd
import numpy as np
import pickle

# importing a vectorizer -> this is how we get to the topic model
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# reading in the data
df = pd.read_json("mod_act_lang_filtered_pre_proc.ndjson", lines=True)

In [3]:
%%time
# loading the pickled model
lda_model = pickle.load(open('mod_act_lda_k10.pk', 'rb'))

CPU times: total: 78.1 ms
Wall time: 98 ms


In [4]:
# function to override the pre-proc that occurs within the vectorizer
# just returns the original string -> because I already had it clean
def dummy_func(x):   
    return x

In [5]:
def cust_tokenizer(x):
    return x.split()

In [6]:
# initializing a TFIDF vectorizer with unigram representation
# it's cheaper and it will allow for relative pruning (those terms appearing within fewer than .05% of docs or in more than 99%)
# feeding in our own functions for splitting and cleaning because otherwise it will mess up our plan
tf_vectorizer = CountVectorizer(
    min_df =.005, max_df = .99,  preprocessor=dummy_func, tokenizer=cust_tokenizer)

In [7]:
%%time
# tf vectorized fiting
tf_vectorizer.fit_transform(df["bo"])

CPU times: total: 2min 29s
Wall time: 2min 29s


<14001320x272 sparse matrix of type '<class 'numpy.int64'>'
	with 126127281 stored elements in Compressed Sparse Row format>

In [8]:
%%time
# vectorized docs
vectorized = tf_vectorizer.transform(df["bo"]) 

CPU times: total: 2min 25s
Wall time: 2min 26s


In [10]:
%%time
# makign the labels
lda_labels =  np.matrix(lda_model.fit_transform(vectorized))

CPU times: total: 10min 23s
Wall time: 1h 30min 25s


In [11]:
%%time
# normalizing the topic distribution
doc_topic_dist = lda_labels/lda_labels.sum(axis=1)

CPU times: total: 609 ms
Wall time: 614 ms


In [12]:
%%time
# code to pull the indices for top topics
df["doc_labels"] = doc_topic_dist.argmax(axis=1)

CPU times: total: 406 ms
Wall time: 403 ms


In [13]:
%%time
# code to pull the proability for each top topic
df["label_probs"] = np.amax(doc_topic_dist, axis=1)

CPU times: total: 859 ms
Wall time: 860 ms


In [14]:
%%time
# getting the value counts here. 
df["doc_labels"].value_counts(normalize=True)*100

CPU times: total: 62.5 ms
Wall time: 72 ms


0    18.456910
4    12.663163
2    12.367105
6    11.633753
5     9.319078
9     8.769845
7     8.640521
1     8.470166
8     5.340704
3     4.338755
Name: doc_labels, dtype: float64

In [15]:
# setting the display limit
pd.set_option('display.max_rows', 200) 

In [16]:
# exploring the 0th (1st) topic
pd.concat([df[(df["doc_labels"]== 0)].head(100), df[(df["doc_labels"]== 0)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
3,p,458f9799ce644843be5a521f9c37c9ac,20191017,1571273910000,84026f1f2aea4e7c930087058c09aefa,bullettdriver,consider,i.imgur.com,English,[0.8588655591000001],0,0.100000
4,p,cbcb980cfc5f434294a2f673274ac7b0,20200715,1594773953000,123a043b86ae4f73ac1e5d55fe849602,ChrisKW,try get real job stop playing victim card pick...,image-cdn.parler.com,English,[0.6118241549],0,0.376051
10,p,f47b99aadf3f4d9cb9b7759ef8788f3e,20201211,1607666218000,84031bab1c524ba7b88edca429e84f34,Nefertari25,year mark 400th anniversary mayflower arriving...,rumble.com,English,[0.9142774343000001],0,0.715241
15,p,cf07975308f04edcb52870de2602c15d,20190729,1564431243000,ba6742827a8e4892b1cb96b083dc0c35,Rockalu,breaking gilroy shooter identified iranianamer...,None,English,[0.8470193744000001],0,0.100000
16,p,7ff4d667bb144782b09d74c5588bc719,20201111,1605055740000,21baaaff87504db0bcfa1f2992715573,DrDogshit58,whistleblower,youtu.be,English,[0.5201298594],0,0.100000
19,p,8bb4a19c184047a3bf523870ff858d19,20201029,1604010763000,0dcc4af8cfdc450881e165ac0a851c34,WarNuse,moment comprehend barry soetoro never america ...,image-cdn.parler.com,English,[0.46391573550000004],0,0.537555
22,p,36627eac32114601a84690b285345895,20200704,1593830167000,f5ed7a0842ad41f2b0856b7092042898,Crossroads,45 magnitude earthquake hit guizhou china rais...,youtube.com,English,[0.7522648573],0,0.774978
25,p,c7747efa75b84364b27211a2bb734a2c,20201128,1606571523000,a29af7c1f68942d3a30c801e8231cda4,KatMan,#ronaldreagan #trumpwillprevail,image-cdn.parler.com,English,[0.25499451160000003],0,0.100000
27,p,ea27b7c4dc7654d84d1b724a195ee2ef,20190105,1546648964000,6737ce255d805d8181667502320158ea,BrandonStraka,anybody would like give follow yet another soc...,None,English,[0.8700723052],0,0.827459
31,p,41debc2fdf5c4c02a5ed47616918ce84,20190608,1559974833000,46c873cd74da439fa585a67c90e59b89,GoGo,tired #winning yet #follow fine people #wwg1wg...,i.imgur.com,English,[0.2918264866],0,0.699932


In [17]:
# exploring the 1st (2nd) topic
pd.concat([df[(df["doc_labels"]== 1)].head(100), df[(df["doc_labels"]== 1)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
13,p,a224618d3e594c3d8874f38de73b872d,20201215,1608071364000,3f8890214152490f941dfc79a329d262,AJCats,police searching bank robber ues manhattan goo...,None,English,[0.9196583033],1,0.411396
23,p,e32446b592fc4f80bd2d6f29186e838b,20200620,1592684751000,dc980ca7eea8474a8349623f48f80862,MrJWesq,bbc news broadcaster chaos broadcaster see low...,express.co.uk,English,[0.9093565345],1,0.819948
47,p,b116b7f15f5e45209322c38a1655229a,20200828,1598635029000,d7ec99ad413856a1ebf3efc34da55a07,JeremyFrankel,biden claim race riot national guard obama lea...,bongino.com,English,[0.672347188],1,0.699995
55,p,1132bd7f3ac54fa985cd8ac4facb7597,20201010,1602350574000,d091dfc81de40f003a9a8560797d926b,Cassandra,joe biden say voter deserve know pack supreme ...,thegatewaypundit.com,English,[0.7342224121],1,0.441390
71,p,81df6fc06a9f4019be0a1363d75c79db,20190716,1563278539000,79faac04de3c420c93564c01af24889f,BBD3,source funding dems leftist radical know else,i.imgur.com,English,[0.9589286447],1,0.699950
75,p,0d6f142559ca40ffbf8c8eadac92d353,20200628,1593364597000,8abb85440ffe48aaa697105ea105f9ef,Blackwelder,fresh evidence obama ordered phony russiagate ...,nypost.com,English,[0.8434334397000001],1,0.549992
82,p,897e095465604826a2aee39cffadb4f9,20200626,1593175981000,dd88da920cfe45c28913085d1e60ed2b,Heaver,week alone three men stabbed bournemouth polic...,westmonster.com,English,[0.9606106877],1,0.549995
90,p,9e8a718f333846a2ba57294179c1831a,20201019,1603134597000,3deea966dd0447c9a98bd9c3d7c88abc,InevitableQ,donald trump reporter joe biden criminal crimi...,None,English,[0.9332408905],1,0.489243
109,p,0bee5656c64b43f2a4010a414b023cd5,20200610,1591821644000,1190d21b3cb443b4905f76ae5d330eb2,TheDamaniFelder,white liberal crack acting like know best peop...,image-cdn.parler.com,English,[0.9934334159],1,0.413852
118,p,9340b01bd5b54c5e9382062cca17a3da,20201209,1607478508000,40d63893f73347abb5b4c085600f6c6e,Hitthegspot,yesterday news china one last major country co...,image-cdn.parler.com,English,[0.8785611391],1,0.585871


In [18]:
# exploring the 2nd (3rd) topic
pd.concat([df[(df["doc_labels"]== 2)].head(100), df[(df["doc_labels"]== 2)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
30,p,e2074b2617004b8eb03fce2404c6ba35,20200703,1593735870000,f5ed7a0842ad41f2b0856b7092042898,Crossroads,#xijinping stated recent politburo meeting top...,None,English,[0.8991113901000001],2,0.419996
45,p,71463cda33f2486b85efcbff20af2450,20201108,1604806279000,74060e06229c45ddb8a50f53d3cedb47,FreedomForceR,might buddy talk ledge video,www.bitchute.com,English,[0.9851741791],2,0.549987
141,p,0ad2f01ae63b471ba0ac58032661a7c2,20200602,1591134095000,725eb0e42bc745c1a6cc7a39dc0178c5,DaveHayes,parler,prayingmedic.com,English,[0.9216817617],2,0.549975
227,p,2b2c49aa4c80464e950c167be03b40f0,20200727,1595882654000,c37d4ca9caa447ac9c667578d725811c,Mac72terry,2016 worked pretty well u #maga2020 #trumptrai...,image-cdn.parler.com,English,[0.7556253672000001],2,0.366657
289,p,b7880d42b0cb48a3a42b063eb1a0e2f5,20201018,1603059020000,c1ab56d9093449409b239080182a29dd,Catherinematthew85,facebook suspends user jamie allmans account p...,thegatewaypundit.com,English,[0.7638020515],2,0.258096
323,p,e6ac1c6681fe4d299fb37b379fb42507,20200817,1597696706000,725eb0e42bc745c1a6cc7a39dc0178c5,DaveHayes,alexander yuk ching 67 former cia officer arre...,justice.gov,English,[0.9874782562000001],2,0.354487
361,p,47c3d8b31cc34c1fbdb59a231bd2c788,20190614,1560475585000,2d2e11f2c2db767cdede1762deda08e8,deplorablekel,lauraloomer nj might worse minnesota michigan ...,deplorablekel.com,English,[0.8467355371],2,0.549999
393,p,f55075a48862495bac2790a585b94f01,20200813,1597283433000,5fe3cd49c88a4b89b321e2e39995e23f,TheVivaFrei,fellow youtube lawyer friend nate broady absol...,None,English,[0.9554314017000001],2,0.366666
575,p,de3685ab9e984e0e956fd19d116d50bb,20201113,1605297144000,ff51a6d600184e00920d436ca2c98019,larryward,parler focus staple street capital owned willi...,None,English,[0.8297603130000001],2,0.549975
588,p,bdd119d2f75447dab1f9b12f2fbdaddc,20210108,1610082189000,5f66376e99d74f7988bbc4c3447a9d6e,TylerrWebster,peep flag put video official team trump instag...,image-cdn.parler.com,English,[0.8039436936000001],2,0.419892


In [19]:
# exploring the 3rd (4th) topic
pd.concat([df[(df["doc_labels"]== 3)].head(100), df[(df["doc_labels"]== 3)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
1,p,a558dea4b7654fe5b9487944e008f53b,20200724,1595611794000,57cc44f72c3646cba8e36421e5a419a5,MeetMichael,city raised let bunch terrorist thug run city ...,foxnews.com,English,[0.9698458314],3,0.659299
8,p,45ff3631d6064a50beb392dca2cf4612,20200803,1596419689000,4ad69e839d1e4a3ea91e0a122bb110c6,CarolYoung2795,join trump train stevenjsargent organized digi...,None,English,[0.47117155790000004],3,0.420303
32,p,921e6628f0f042be889e1b0014216d51,20201114,1605312685000,e44e8f8c5fb94a868aafe6843eae98fb,SomeBitchIKnow,please pray,None,English,[0.9787321091000001],3,0.549993
50,p,f6da679ed2a44868a35e616a76eba225,20201224,1608818084000,b638877c7eae4e7299796872fad8c5c8,TurdFergusonx,follow back patriot,image-cdn.parler.com,English,[0.7028267384],3,0.774970
69,p,9ed32a2a28b04971955894c63f07c626,20201103,1604421680000,510087606bc346c8a20db1557ebdf79e,realMikeLindell,medium come everyone get vote reach everyone k...,image-cdn.parler.com,English,[0.9697633982],3,0.408008
76,p,7b4510d2c0f64b1ca634641f1a836f75,20201114,1605378361000,1309e3892a444462bc60d71c39ab499b,RamblingRuss,urgent message antifa blm put immediate lid da...,None,English,[0.9124426842000001],3,0.557770
81,p,1fabbc5bbae446c79e0d21dd0be2224f,20200827,1598571018000,41f71319fcb947429a094b6ea93c9b26,XPSD,adding new product daily everything buy site g...,xpsd.news,English,[0.7819012999999999],3,0.729768
94,p,d9b71be4b8fe481c988c3db20a6e8382,20201225,1608874765000,a606dc63dd4d4c48a57cf01309c7758c,JV3MRC,merry christmas everyone parlerverse,c.tenor.com,English,[0.7743878961],3,0.549986
97,p,a1f19dd107014bd299aa6659550e1ccd,20200531,1590938086000,85e2c55fa4c14d7ea388176d73e807a9,Daryus,peaceful protester exposing antifa terroist #a...,None,English,[0.2865837812],3,0.549995
116,p,970a69935a2341c89a982e146bf245de,20201021,1603289547000,8cb5b7845401487192c83c726d51b64a,SimplySoaring,please sign thanks,recallgavin2020.com,English,[0.9213376045],3,0.699990


In [20]:
# exploring the 4th (5th) topic
pd.concat([df[(df["doc_labels"]== 4)].head(100), df[(df["doc_labels"]== 4)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
2,p,986a1719672d4deb99140993c94be6bf,20201112,1605196368000,ac6b1de1f488f8883d33151405b078dc,azws,built custom glockinc 19 amazing gf miaaz13 hu...,image-cdn.parler.com,English,[0.5646400452],4,0.664563
93,p,c9496407af6b44d7a42de4821917e451,20200727,1595866820000,039c41b8fe4743c090ce24c064d647ce,Megsays,joking right,image-cdn.parler.com,English,[0.4448182285],4,0.549986
110,p,1e90a1d219f34c4eb63e2c509525d834,20201210,1607609131000,2a21461a89c6430e9ef9f74f6c4343cb,Jeeper43215,ghandi assassinated like doctor martin luther ...,image-cdn.parler.com,English,[0.8633802533],4,0.549925
267,p,c88ee523ad3a4cd0998753551b2b500d,20200905,1599349775000,825ee061818e87be763f3853af3ab7ad,Michael,thought,chaospost.com,English,[0.9705010056000001],4,0.549997
663,p,ecf17551b6f9421dabae41bdfec1a841,20201102,1604317597000,a9dcee03a0da41fa864e79a7279832e4,Richvaldes,bernie sander endorses farleft portland mayor ...,theblaze.com,English,[0.7724239826],4,0.366667
698,p,ad642ba458014b60aecf439bdc407621,20200516,1589666756000,6b6eb66e72884d059e3327b0a67d9b61,Jfred1957,right,image-cdn.parler.com,English,[0.9864090085],4,0.549986
829,p,82cc7f5b0d184ec39f305bc041f6ab95,20201015,1602804760000,c4502c0b435d42a7a7483f9d760cbb6b,ConnieHair,special #backtheblue,washingtontimes.com,English,[0.9509575367],4,0.550000
953,p,9193b5bdc0bf43d3b41f6df60d249a37,20201211,1607715954000,90b288d835eb4763ba72dd5d6988f44f,Owenshroyer,thought,image-cdn.parler.com,English,[0.9237470031],4,0.549997
963,p,2b07d89127084c4b93014c31e7872bcd,20200606,1591452497000,ff51a6d600184e00920d436ca2c98019,larryward,angry division anarchy respectful division lib...,None,English,[0.8587437272],4,0.550000
1017,p,2ffc1b0c469340d39de374545080673c,20201116,1605563749000,68c766f6a0e6430cbd31a415fcb6e4fd,Jenperez133,new tweet linwood,image-cdn.parler.com,English,[0.5570518374],4,0.549979


In [21]:
# exploring the 5th (6th) topic
pd.concat([df[(df["doc_labels"]== 5)].head(100), df[(df["doc_labels"]== 5)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
56,p,f43d1e7df1c740af9c758567966e8b2b,20201231,1609404239000,e28b6d3669794efe915467f250e05967,Wjmccartney,leave tonight gem,image-cdn.parler.com,English,[0.7246234417],5,0.549994
147,p,9f2e2d876b044e0287f8759b218383da,20201018,1602993830000,60f2f27b128148959b7f73d0114efbbb,TIG13,pa celebrating america pride #alwaysmoveforwar...,image-cdn.parler.com,English,[0.6047005653],5,0.549989
303,p,67628d29dc034514951cbcd2ae750cc2,20201116,1605557795000,22110fccc3014b0fba6864d944662314,frcdc,prolife issue winning america #prolife #conser...,christianpost.com,English,[0.7436894178],5,0.549989
329,p,9a51b9fac9444f5b92fc02124f03c8ed,20190514,1557872888000,a078092e6e2df507bfd12db471029eb3,John,interested la vega presenting code class tonig...,None,English,[0.8181661963],5,0.524152
375,p,bf7def5a6a0b4e86b719ecab9b0909d4,20201225,1608865849000,1d9b6ed28de04575a2b85bce0575216b,aaronchildress,got dick pic parler accident meant send boe ji...,None,English,[0.9921330214],5,0.495790
455,p,69890030e0bf42e2a3c0e7ef30cad532,20200117,1579274907000,789ed245569b4b47b0fa97893c7e4dc7,MikeU,make happen,image-cdn.parler.com,English,[0.8186787963000001],5,0.549987
516,p,029ee0f656fe447cb1fe28bba11dfa26,20201206,1607286538000,c6ba84c70b464e0ba83ae44c101fb430,beholdisrael,admire america mayor rudi giuliani,image-cdn.parler.com,English,[0.6607938409],5,0.549989
540,p,a571ca2916794c6dbbf77961b0f90477,20200817,1597626375000,2ea66900ed69a5005fe51fc8b0881f93,WatchChad,sure subscribe listen rate,podcasts.apple.com,English,[0.8723914623000001],5,0.549999
579,p,46a2fcc724f344898d45a63779e555f9,20190612,1560297770000,a078092e6e2df507bfd12db471029eb3,John,great meet congressman crenshaw,i.imgur.com,English,[0.8654345870000001],5,0.549998
580,p,6f0bb3b8735545c3b90bed4f4c9932b5,20200818,1597773749000,e77148c861684396afb11600f6871283,verdeeamarela,corumbms 18082020 link youtube,youtu.be,English,[0.2422387004],5,0.366667


In [22]:
# exploring the 6th (7th) topic
pd.concat([df[(df["doc_labels"]== 6)].head(100), df[(df["doc_labels"]== 6)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
14,p,8cf55e03be894878ae263d1ff201cd61,20200923,1600897959000,11054c91f57d4dd192d2ef8af3b3eb1d,SharylAttkisson,filling scotus seat poll vote go sharylattkiss...,image-cdn.parler.com,English,[0.8271412849],6,0.351520
34,p,87dc12fbb1d94674baa10c4e9aeda6c4,20201120,1605861330000,f8de2bbaca804499ad7ac6e00d339d80,Cynthiadmail,need stop madness people top canadian patholog...,thegatewaypundit.com,English,[0.8204674125],6,0.887477
35,p,da721d68fbd34554850dee6ca9fdd0eb,20200103,1578019498000,f1c6851c95c98c2d2af35eec0cb7b845,mflynnJR,say iran going retaliate iranian general would...,None,English,[0.977627039],6,0.552455
36,p,5fee8b207a8e430798eb8b4b07ce4823,20201202,1606937021000,7714bc7dc2684c9794d94cf5b423d6df,Dmak2001,starting get sense whole government full chine...,media1.giphy.com,English,[0.9086901546],6,0.509477
42,p,523cb93ba15345a481ffde2d3f2f9cc2,20201210,1607620998000,903399ea109448d89efe70edd185047f,TomBreaksNews,infowars vaccine whistleblower found dead sayi...,infowars.com,English,[0.43982449170000004],6,0.538939
44,p,9a59638a654d4562a555fc3fd1653821,20200818,1597794276000,d804bfb432be44e4bb6b40fe9880dc74,Randy1usa,watch fox one far left make u madder hell leav...,image-cdn.parler.com,English,[0.9233920574000001],6,0.335039
51,p,f7ebd26dadf54e32b5352278280ac7ba,20200808,1596905478000,2fc23d937dea45d6b3c13e389f86a1e2,Bobbychurchill59,democrat msm say protest peaceful yea right li...,youtu.be,English,[0.9852642417],6,0.324102
57,p,0c4e391569f64526a454421399385261,20201125,1606268643000,584c17668ac942f39596325387d92d11,Rosababy1111,million year would think would effect thought ...,image-cdn.parler.com,English,[0.9712108374],6,0.463697
62,p,10a90992c42e43bf80f2eddb56934151,20210107,1610049298000,575f27c65488498386208c14742c21a9,Ksorbs,via theblaze problem antifa burning business k...,theblaze.com,English,[0.9258968234],6,0.727707
74,p,942e0ddd92a44ddc82cb59f211cbf98b,20201208,1607398390000,47dcba0ede6746aebf74075df2357fe3,Barbosa,bill melugin fox la reporting la county da geo...,None,English,[0.8824691772000001],6,0.448503


In [23]:
# exploring the 7th (8th) topic
pd.concat([df[(df["doc_labels"]== 7)].head(100), df[(df["doc_labels"]== 7)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
5,p,bdc2b02893a74de1897ab60a952d92e5,20210103,1609710557000,f6f720cab24540c7b562dc4d61da78b2,Eloyburn,idiot like paul ryan act like high school mad ...,image-cdn.parler.com,English,[0.9828255177],7,0.467063
6,p,81ab19398ce34bdabe38dd44fe0f8073,20201231,1609456822000,0791b0260cd04c76938ddaff1e29a8dc,Tizimal1,amazing steve bannons fight trump speech early...,thegatewaypundit.com,English,[0.8558598757],7,0.442405
11,p,a464a5c71b4d43cda5c082f1704269d5,20190608,1560028330000,5908787cae924dbab06fcfb9ae91a880,AverageAmerican,fake news cnn want promote liberal downplay co...,None,English,[0.9406405091000001],7,0.485685
18,p,b945d40f72e74c3aad058ef1867b6602,20201216,1608145371000,5f660df865bc4b15bc8fde3009b49938,SaraCarterOfficial,#biden say confident son #hunterbiden nothing ...,saraacarter.com,English,[0.9534426332],7,0.409388
26,p,05f282f4b56f44eb8182818edb53d013,20190615,1560565961000,5953e9f83c1f4f488dc6c9f98a4a95ce,LizWheeler,august 6th hurry get cannot wait read book wan...,amazon.com,English,[0.8675927520000001],7,0.361719
33,p,b9b982e877e04ff59f8245442d2a46ab,20191117,1574012044000,b95c6d9b97cd4cfb99483e6e2df962aa,DashRiprock,blocked still program top 25 rating would virt...,i.imgur.com,English,[0.9483394623],7,0.887479
52,p,8512a12ef15d43259f721c6c211e7cf7,20201105,1604620544000,6fa3c4b62b86486481c794bfe4e797b1,kingofdangle,zero wuhan flu coverage last 48 hour sohow isg...,None,English,[0.8489345312000001],7,0.699977
54,p,25c8a8a8ac364df98f339a829b42f08f,20200627,1593230650000,408d2fae467e4a46a49fbb765c1272ed,Mollywood,taking america back,image-cdn.parler.com,English,[0.7354748249],7,0.409571
59,p,bd57dcdf37234197af04552aa579e1eb,20190623,1561274304000,5e94d4ec92ef43a881ff8e0db1948ca3,JenniferGarrett,medium never show trump supporter praying rall...,i.imgur.com,English,[0.8898857832],7,0.605340
72,p,9e5f898897f34d578b82f98416083d4c,20200628,1593311156000,8bf5b25eb8934888929d3b07ad7321f3,TheSaltyCracker,salty officially parler yet something working ...,None,English,[0.9910411835],7,0.370843


In [24]:
# exploring the 8th (9th) topic
pd.concat([df[(df["doc_labels"]== 8)].head(100), df[(df["doc_labels"]== 8)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
21,p,2945ed534ada4adbba7d868e9befe477,20190613,1560399845000,b23f5402c1b64f4a9f4ec3cba46e3c2b,OldManDiogenes,putting next train together know many follow m...,i.imgur.com,English,[0.9766044021],8,0.533314
37,p,7986d812b2274c298d1e25d02e3547a1,20200627,1593243349000,57a939db6aef4669b4b13a54eae8421d,DarrenPlymouth,good morning parler people fucking saturday me...,youtube.com,English,[0.8394244313],8,0.409901
49,p,21bb95a0746d4cfa8a6f8ed2ab113aed,20200721,1595348347000,b4aafe0cf091473d83a7c3b3e3831651,Freebricksnelle,nigga see unblock twitter 5 year lmaooooo,None,English,[0.8724549413],8,0.523260
64,p,6db55748dfbc4badac99f566f6bbeb70,20200628,1593326188000,3d555a5a98174778aec58cc3c0726a94,SenMikeLee,worship government idolatry time,None,English,[0.9851661325000001],8,0.371119
70,p,8136b36f89014c9dbe79d6f60e38a3bb,20200627,1593254305000,138335f5e42c46b6b9cf52a5c295d3f7,AlecNorton,well said kate hoey,image-cdn.parler.com,English,[0.9293174148000001],8,0.699974
78,p,daabcca5971f4962b434a5b645f2bfa9,20200728,1595928494000,7c251063f4444eebb2261af853054239,DigitalSoldier1972,people sick,image-cdn.parler.com,English,[0.9898255467],8,0.549942
79,p,3073130413ea4804af5517ac0e936cfd,20201106,1604644403000,1190d21b3cb443b4905f76ae5d330eb2,TheDamaniFelder,liberal state start take turn,image-cdn.parler.com,English,[0.7976008654000001],8,0.330485
103,p,3093c9022b4b47ac83c7e8668bb8a9fa,20200724,1595608374000,be5d618a72644da29b59b212d15bdec4,EscapeGoated,tried several time upload maybe work time #joe...,None,English,[0.9805513024],8,0.587978
108,p,2712ce898a13476f8fef12f5be3a7a6c,20200709,1594279862000,383feb939f898fb5017bd067629fdace,ablair,good evening everyone #parler #latenight check...,None,English,[0.9624277353],8,0.391133
139,p,851f6991801241319752aad64246652b,20190618,1560875224000,199ab7d2600945fe92521ca7f527e50d,ScottyGovoni,irs fraud upon american people long past time ...,foxnews.com,English,[0.9812015295000001],8,0.484638


In [25]:
# exploring the 9th (10th) topic
pd.concat([df[(df["doc_labels"]== 9)].head(100), df[(df["doc_labels"]== 9)].tail(100)],axis=0)

,t,id,cd,c,u,un,bo,dmn,bo_lang,conf,doc_labels,label_probs
0,p,67151e44fbd741ee90607e8327399bec,20210106,1609909302000,cd091080202946ce9028784afb84d08a,Civilright,zero punishment election cheating saw georgia ...,None,English,[0.9863173366],9,0.849973
7,p,db92a291771c44a3970e8e9d5d33dcdc,20201108,1604800646000,65af37d393584e04893350b0f46259b9,TheDonaldrefugee,asshat al gore also inaugurated president medi...,image-cdn.parler.com,English,[0.9912668467],9,0.479881
9,p,b7bc40090ce04e7f9b33b7d5b2c6945c,20210103,1609688627000,c4502c0b435d42a7a7483f9d760cbb6b,ConnieHair,briefing yesterday republican state legislator...,theepochtimes.com,English,[0.9874636531000001],9,0.849987
12,p,1950bcd00ff84835b9d7664262f89089,20200727,1595813001000,aa85d8881b794fd2b44ea0e61bc4ff34,KayleighMcEnany,tragedy gotten zero coverage black trump suppo...,google.com,English,[0.9728655815],9,0.379751
17,p,254939aa01aa4b5686c073c2b13bc7dc,20201220,1608498925000,368284fc02324b7b971b3eca43fc4148,DalBeck45th,13 million vote registered voter people global...,thegatewaypundit.com,English,[0.866623342],9,0.590956
20,p,ab0a3ab2a2c741fbb844e151e9d34dec,20201119,1605820980000,cc8e648f96ce49ccbf8f96cfce81725f,Sharmccut,explanation governor kemp secretary state ga b...,None,English,[0.9817851186000001],9,0.819999
24,p,d30f1d50a1844707bd3f1c9eb8eb96dc,20200731,1596164446000,b7f6f7383934c84f664ad3d93dc31ca6,carpedonktum,carpes corner episode 00013 milo matt removed ...,None,English,[0.9335160255],9,0.241637
28,p,a66f5b6001a3402faccd08a5c6464ec3,20200628,1593347434000,06f26fc11f78406ebffc27cd88e48ab7,Mckaylarosej,downfall republican party compliance,None,English,[0.9482269287],9,0.699972
29,p,7180e1d49b6a42b4a4af86d69dad7099,20200807,1596833607000,e2bc8575aa3d4112934e92ff7fdcc165,JessBowling,trump win america win suck one pelosi get exec...,foxbusiness.com,English,[0.8831784129],9,0.899977
39,p,873be179ea1640cd8d122c043daf10ec,20190530,1559258932000,ef8808c659414c5aaa7afebe139e48bd,theRealConservativelatino,question american stand one administration try...,None,English,[0.9337733984000001],9,0.643267
